У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

# Taks 1 solution

## Imports and general review

In [2]:
import pandas as pd
import numpy as np

raw_df = pd.read_csv('./data/customer_segmentation_train.csv', index_col='ID')
raw_df.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
ID,,,,,,,,,,
462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [3]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8068 entries, 462809 to 461879
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           8068 non-null   object 
 1   Ever_Married     7928 non-null   object 
 2   Age              8068 non-null   int64  
 3   Graduated        7990 non-null   object 
 4   Profession       7944 non-null   object 
 5   Work_Experience  7239 non-null   float64
 6   Spending_Score   8068 non-null   object 
 7   Family_Size      7733 non-null   float64
 8   Var_1            7992 non-null   object 
 9   Segmentation     8068 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 693.3+ KB


In [4]:
from mlpeople.eda.reporting import describe_dataframe

describe_dataframe(raw_df)

,column_id,data_type,unique_values,missing_values,mean,std,min,max
0,Gender,object,2,0,NaN,NaN,NaN,NaN
1,Ever_Married,object,2,140,NaN,NaN,NaN,NaN
2,Age,int64,67,0,43.47,16.71,18.0,89.0
3,Graduated,object,2,78,NaN,NaN,NaN,NaN
4,Profession,object,9,124,NaN,NaN,NaN,NaN
5,Work_Experience,float64,15,829,2.64,3.41,0.0,14.0
6,Spending_Score,object,3,0,NaN,NaN,NaN,NaN
7,Family_Size,float64,9,335,2.85,1.53,1.0,9.0
8,Var_1,object,7,76,NaN,NaN,NaN,NaN
9,Segmentation,object,4,0,NaN,NaN,NaN,NaN


## Missing values

In [5]:
from mlpeople.eda.missing import get_null_df

get_null_df(raw_df)

,column_name,null_count,null_percentage
5,Work_Experience,829,10.28
7,Family_Size,335,4.15
1,Ever_Married,140,1.74
4,Profession,124,1.54
3,Graduated,78,0.97
8,Var_1,76,0.94
0,Gender,0,0.00
2,Age,0,0.00
6,Spending_Score,0,0.00
9,Segmentation,0,0.00


## Split data

In [6]:
target_col = "Segmentation"

In [7]:
from sklearn.model_selection import train_test_split

def split_train_test_df(df, target_col, test_size=0.2, stratify_by_target=True, random_state=42):
    X = df.drop(columns=target_col)
    y = df[target_col]
    
    stratify = y if stratify_by_target else None

    return train_test_split(
        X,
        y,
        test_size=test_size,
        random_state=random_state,
        stratify=stratify
    )
    

X_train, X_test, y_train, y_test = split_train_test_df(raw_df, target_col, test_size=0.2, random_state=42)

## Pipeline

In [8]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Selectors
numeric_selector = make_column_selector(dtype_include=np.number)
categorical_selector = make_column_selector(dtype_exclude=np.number)

# create function and params for experimenting possibilities
def get_model_pipeline(
    model=None,
    num_impute_strategy="most_frequent",
    num_impute_value=0, # applicable only if num_impute_strategy="constant"
    cat_impute_strategy="most_frequent",
    cat_impute_value="missing", # applicable only if cat_impute_strategy="constant"
    cat_encoder_drop=None
):
    if model is None:
        model = LogisticRegression(solver='liblinear')

    # Numeric pipeline
    num_imputer_params = {"strategy": num_impute_strategy}
    if num_impute_strategy == "constant":
        num_imputer_params["fill_value"] = num_impute_value

    numeric_pipeline = Pipeline([
        ("imputer", SimpleImputer(**num_imputer_params)),
        ("scaler", StandardScaler())
    ])

    # Categorical pipeline
    cat_imputer_params = {"strategy": cat_impute_strategy}
    if cat_impute_strategy == "constant":
        cat_imputer_params["fill_value"] = cat_impute_value

    categorical_pipeline = Pipeline([
        ("imputer", SimpleImputer(**cat_imputer_params)),
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop=cat_encoder_drop))
    ])

    # Combine
    preprocessor = ColumnTransformer([
        ("num", numeric_pipeline, numeric_selector),
        ("cat", categorical_pipeline, categorical_selector),
    ])

    pipeline = Pipeline([
        ("preprocessing", preprocessor),
        ("model", model)
    ])

    return pipeline


In [9]:
model_pipeline = get_model_pipeline()
model_pipeline

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers c

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

# Task 2 Solution
## SMOTENC

In [10]:
from imblearn.over_sampling import SMOTENC

# categorical_selector
categorical_cols = X_train.select_dtypes(exclude=np.number).columns

# SMOTENC requires **column indices**, not names
categorical_indices = [X_train.columns.get_loc(col) for col in categorical_cols]

smote = SMOTENC(
    categorical_features=categorical_indices, 
    sampling_strategy='auto',
    random_state=42
)

# SMOTENC can not handle missing values - use preprocessors from pipeline
preprocessed_X = model_pipeline.named_steps["preprocessing"].fit_transform(X_train)

X_SMOTENC_resampled, y_SMOTENC_resampled = smote.fit_resample(preprocessed_X, y_train)


In [11]:
y_train.value_counts()

Segmentation
D    1814
A    1578
C    1576
B    1486
Name: count, dtype: int64

In [12]:
y_SMOTENC_resampled.value_counts()

Segmentation
A    1814
B    1814
C    1814
D    1814
Name: count, dtype: int64

## SMOTETomek

In [13]:
from imblearn.combine import SMOTETomek

smotetomek = SMOTETomek(random_state=42)
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(
    preprocessed_X,
    y_train
)

In [14]:
y_train_smotetomek.value_counts()

Segmentation
C    1503
D    1487
B    1455
A    1433
Name: count, dtype: int64

In [15]:
y_train.value_counts()

Segmentation
D    1814
A    1578
C    1576
B    1486
Name: count, dtype: int64

## New Pipeline with imblearn.pipeline

In [16]:
from imblearn.pipeline import Pipeline as Imblearn_Pipeline
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek


numeric_selector = make_column_selector(dtype_include=np.number)
categorical_selector = make_column_selector(dtype_exclude=np.number)


def get_model_pipeline_resample_ext(
    model=None,
    sampler=None,  # None | "smotenc" | "smotetomek"
    num_impute_strategy="most_frequent",
    cat_impute_strategy="most_frequent",
    cat_encoder_drop=None,
    random_state=42,
):

    if model is None:
        model = LogisticRegression(solver='liblinear')

    # IMPUTERS first before resampling to avoid missing
    numeric_imputer = SimpleImputer(strategy=num_impute_strategy)
    categorical_imputer = SimpleImputer(strategy=cat_impute_strategy)

    imputer = ColumnTransformer([
        ("num", numeric_imputer, numeric_selector),
        ("cat", categorical_imputer, categorical_selector),
    ])

    # ENCODER + SCALER
    # after resampling in case of SMOTENC,
    # before resampling in case of SMOTETomek as it does not support categorical values handling
    postprocessor = ColumnTransformer([
        ("num", StandardScaler(), numeric_selector),
        ("cat", OneHotEncoder(
            handle_unknown="ignore",
            drop=cat_encoder_drop
        ), categorical_selector),
    ])

    steps = [
        ("imputer", imputer),
    ]

    # SAMPLER
    if sampler == "smotenc":
        # Need categorical indices AFTER imputation but BEFORE encoding.
        # ColumnTransformer keeps order: numeric first, categorical next.
        sampler_step = SMOTENC(
            categorical_features="auto",
            random_state=random_state
        )

        steps.append(("sampler", sampler_step))
        steps.append(("postprocess", postprocessor))

    elif sampler == "smotetomek":

        # Must encode BEFORE sampling
        steps.append(("postprocess", postprocessor))

        sampler_step = SMOTETomek(random_state=random_state)
        steps.append(("sampler", sampler_step))

    else:
        steps.append(("postprocess", postprocessor))

    steps.append(("model", model))

    return Imblearn_Pipeline(steps)


In [17]:
model_imblearn_pipeline = get_model_pipeline_resample_ext(sampler="smotenc")
model_imblearn_pipeline

,steps,"[('imputer', ...), ('sampler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The gi

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?